After creating the database the next step is to create the automatitazion of downloading data from yfinance and finvizfinance and store it in the database

In [36]:
from sqlalchemy import create_engine
import pandas as pd
import yfinance as yf
from finvizfinance.quote import finvizfinance

import os
from sqlalchemy import create_engine, text
from dotenv import load_dotenv

from urllib.parse import quote # If the password contains weird letter


In [37]:
def connect_db():


    load_dotenv()

    USER = os.getenv("DB_USER")
    PASSWORD = quote(os.getenv("DB_PASSWORD"))
    HOST = os.getenv("DB_HOST")
    PORT = os.getenv("DB_PORT")
    DB_NAME = os.getenv("DB_NAME")
    

    engine = create_engine(f"postgresql://{USER}:{PASSWORD}@{HOST}:{PORT}/{DB_NAME}")
    return engine






In [38]:
connect_db

<function __main__.connect_db()>

In [39]:
import psycopg2

load_dotenv()

USER = os.getenv("DB_USER")
PASSWORD = os.getenv("DB_PASSWORD")
HOST = os.getenv("DB_HOST")
PORT = os.getenv("DB_PORT")
DB_NAME = os.getenv("DB_NAME")

try:
    conn = psycopg2.connect(
        dbname="portfolio_optimizer",
        user="postgres",
        password=PASSWORD,  # Reemplaza con tu contraseña real
        host="127.0.0.1",  # Prueba con 127.0.0.1 en vez de localhost
        port="5432"
    )
    print("✅ Connection successful!")
    conn.close()
except Exception as e:
    print(f"❌ Connection failed: {e}")


✅ Connection successful!


In [40]:
from sqlalchemy import create_engine, text

load_dotenv()

USER = os.getenv("DB_USER")
PASSWORD = quote(os.getenv("DB_PASSWORD"))
HOST = os.getenv("DB_HOST")
PORT = os.getenv("DB_PORT")
DB_NAME = os.getenv("DB_NAME")

try:
    engine = create_engine(f"postgresql+psycopg2://{USER}:{PASSWORD}@{HOST}:{PORT}/{DB_NAME}")
    with engine.connect() as conn:
        conn.execute(text("SELECT 1"))
    print("✅ SQLAlchemy connection successful!")
except Exception as e:
    print(f"❌ SQLAlchemy connection failed: {e}")


✅ SQLAlchemy connection successful!


# Other functions

In [ ]:
## Funxtion needed to eliminate the sublevels and multindex dataframe to insert data coirrectly. For example in Historical prices
def preprocess_stock_data(stock_data):
    stock_data.columns = stock_data.columns.droplevel(1)

    stock_data.columns.name = None

    stock_data.index.name = None

    return stock_data

# analyst_recommendations

In [41]:
def store_analyst_recommendations(ticker):
    engine = connect_db()
    
    dat = yf.Ticker(ticker)
    
    df = pd.DataFrame([dat.analyst_price_targets])
    
    df['Date'] = pd.Timestamp.today()
    df["ticker"] = ticker
    df = df.rename(columns={"Date": "date", "current": "latest_stock_price", "high": "high_target",
                            "low": "low_target", "mean": "mean_target", "median": "median_target"})
    
    df.to_sql("analyst_recommendations", engine, if_exists="append", index=False)
    print(f"Data{ticker} inserted in analyst_recommendations.")



def delete_analyst_recommendations(ticker):
    engine = connect_db()
    with engine.connect() as conn:
        conn.execute(text("DELETE FROM analyst_recommendations WHERE ticker = :ticker"), {"ticker": ticker})
        conn.commit()
    print(f"Datos históricos de {ticker} eliminados de analyst_recommendations.")

# analyst_recommendations_period

In [42]:
def store_analyst_recommendations_period(ticker):
    engine = connect_db()
    
    dat = yf.Ticker(ticker)

    recommendations = dat.recommendations_summary
    if recommendations is None:
        print(f"No hay datos de recomendaciones de analistas para {ticker}.")
        return

    df = pd.DataFrame([recommendations])

    df["ticker"] = ticker
    df["date"] = pd.Timestamp.today()


    df = df.rename(columns={
        "strongBuy": "strong_buy",
        "buy": "buy",
        "hold": "hold",
        "sell": "sell",
        "strongSell": "strong_sell"
    })

    df.to_sql("analyst_recommendations_period", engine, if_exists="append", index=False)
    print(f"Datos de {ticker} insertados en analyst_recommendations_period.")


def delete_analyst_recommendations_period(ticker):
    """Elimina las recomendaciones de analistas de un ticker específico en la base de datos."""
    engine = connect_db()
    with engine.connect() as conn:
        conn.execute(text("DELETE FROM analyst_recommendations_period WHERE ticker = :ticker"), {"ticker": ticker})
        conn.commit()
    print(f"Datos de {ticker} eliminados de analyst_recommendations_period.")

# Company info

In [43]:
def store_company_info(ticker):

    engine = connect_db()
    stock = yf.Ticker(ticker)
    info = stock.info
    
    df = pd.DataFrame([{
        "ticker": ticker,
        "name": info.get("longName", ""),
        "sector": info.get("sector", ""),
        "industry": info.get("industry", ""),
        "market_cap": info.get("marketCap"),
        "enterprise_value": info.get("enterpriseValue"),
        "total_revenue": info.get("totalRevenue"),
        "ebitda": info.get("ebitda"),
        "net_income": info.get("netIncomeToCommon"),
        "total_debt": info.get("totalDebt"),
        "free_cashflow": info.get("freeCashflow"),
        "return_on_assets": info.get("returnOnAssets"),
        "return_on_equity": info.get("returnOnEquity"),
        "trailing_pe": info.get("trailingPE"),
        "forward_pe": info.get("forwardPE"),
        "price_to_sales": info.get("priceToSalesTrailing12Months"),
        "price_to_book": info.get("priceToBook"),
        "dividend_rate": info.get("dividendRate"),
        "dividend_yield": info.get("dividendYield"),
        "payout_ratio": info.get("payoutRatio"),
        "shares_outstanding": info.get("sharesOutstanding"),
        "held_percent_institutions": info.get("heldPercentInstitutions"),
        "held_percent_insiders": info.get("heldPercentInsiders"),
        "beta": info.get("beta"),
        "fifty_two_week_high": info.get("fiftyTwoWeekHigh"),
        "fifty_two_week_low": info.get("fiftyTwoWeekLow"),
        "earnings_growth": info.get("earningsGrowth"),
        "revenue_growth": info.get("revenueGrowth"),
        "profit_margins": info.get("profitMargins"),
        "gross_margins": info.get("grossMargins"),
        "operating_margins": info.get("operatingMargins"),
        "ebitda_margins": info.get("ebitdaMargins"),
        "target_high_price": info.get("targetHighPrice"),
        "target_low_price": info.get("targetLowPrice"),
        "target_mean_price": info.get("targetMeanPrice"),
        "recommendation_mean": info.get("recommendationMean"),
        "recommendation_key": info.get("recommendationKey"),
        "number_of_analyst_opinions": info.get("numberOfAnalystOpinions"),
    }])


    df.to_sql("company_info", engine, if_exists="append", index=False)
    print(f"Datos de {ticker} insertados en company_info.")





def delete_company_info(ticker):
    engine = connect_db()
    with engine.connect() as conn:
        conn.execute(text("DELETE FROM company_info WHERE ticker = :ticker"), {"ticker": ticker})
        conn.commit()
    print(f"Información de {ticker} eliminada de company_info.")

# funds_holding

In [44]:


def store_funds_holding(ticker, fund="SPY"):
    engine = connect_db()
    
    spy= yf.Ticker('SPY').funds_data
    df = spy.top_holdings
    
    df = df.reset_index()
    df['Date'] = pd.Timestamp.today()
    df["fund_ticker"] = ticker
    df = df.rename(columns={"Date": "date_reported", "Holding Percent": "holding_percent"})
    
    df.to_sql("funds_holding", engine, if_exists="append", index=False)
    print(f"Data{ticker} inserted in funds_holding.")



def delete_funds_holding(ticker):
    engine = connect_db()
    with engine.connect() as conn:
        conn.execute(text("DELETE FROM funds_holding WHERE ticker = :ticker"), {"ticker": ticker})
        conn.commit()
    print(f"Datos históricos de {ticker} eliminados de funds_holding.")

# Historical prices

In [69]:
def store_historical_prices(ticker, period_given="5y"):
    engine = connect_db()
    
    df = yf.download(ticker, period=period_given)
    
    df = df.reset_index()
    df["ticker"] = ticker

    df = df.rename(columns={"Date": "date", "Open": "open_price", "High": "high_price",
                            "Low": "low_price", "Close": "close_price",
                            "Volume": "volume"})
    df=preprocess_stock_data(df)

    df.to_sql("historical_prices", engine, if_exists="append", index=False)

    
    print(f"Data{ticker} inserted in historical_prices.")



def delete_historical_prices(ticker):
    engine = connect_db()
    with engine.connect() as conn:
        conn.execute(text("DELETE FROM historical_prices WHERE ticker = :ticker"), {"ticker": ticker})
        conn.commit()
    print(f"Datos históricos de {ticker} eliminados de historical_prices.")

# institutional_shareholders

In [46]:
def store_institutional_shareholders(ticker):
    engine = connect_db()

    stock = yf.Ticker(ticker)
    
    if not hasattr(stock, "institutional_holders") or stock.institutional_holders is None:
        print(f"No se encontraron datos de accionistas institucionales para {ticker}.")
        return

    df = stock.institutional_holders.copy()
    
    df["date_reported"] = pd.Timestamp.today().date()
    df["ticker"] = ticker
    df = df.rename(columns={"Holder": "holder", "% Out": "pct_held", "Shares": "shares", 
                            "Value": "value", "Pct Change": "pct_change"})

    df = df[["ticker", "date_reported", "holder", "pct_held", "shares", "value", "pct_change"]]

    df.to_sql("institutional_shareholders", engine, if_exists="append", index=False)
    print(f"Data for {ticker} inserted in institutional_shareholders.")


def delete_institutional_shareholders(ticker):
    engine = connect_db()
    with engine.connect() as conn:
        conn.execute(text("DELETE FROM institutional_shareholders WHERE ticker = :ticker"), {"ticker": ticker})
        conn.commit()
    print(f"Datos históricos de {ticker} eliminados de institutional_shareholders.")

# News

In [47]:
def store_news_data(ticker):
    engine = connect_db()
    
    stock = finvizfinance(ticker)
    news_df = stock.ticker_news()
    
    news_df = news_df.reset_index()
    news_df["ticker"] = ticker
    news_df = news_df.rename(columns={"Date": "date", "Title":"headline", "Link":"link", "Source":"source"})
    
    news_df.to_sql("news_data", engine, if_exists="append", index=False)
    print(f"Data{ticker} inserted in news_data.")

def delete_news_data(ticker):
    engine = connect_db()
    with engine.connect() as conn:
        conn.execute(text("DELETE FROM news_data WHERE ticker = :ticker"), {"ticker": ticker})
        conn.commit()
    print(f"Data from {ticker} eliminated from news_data.")

# options_data

In [48]:
def store_options_data(ticker):
    engine = connect_db()

    stock = yf.Ticker(ticker)
    
    try:
        options_dates = stock.options  
        all_options = []

        for date in options_dates:
            opt_chain = stock.option_chain(date)
            calls = opt_chain.calls
            puts = opt_chain.puts

            calls["option_type"] = "call"
            puts["option_type"] = "put"

            df = pd.concat([calls, puts])

            df["ticker"] = ticker
            df["last_trade_date"] = pd.to_datetime(df["lastTradeDate"])
            df = df.rename(columns={
                "contractSymbol": "contract_symbol",
                "strike": "strike",
                "lastPrice": "last_price",
                "bid": "bid",
                "ask": "ask",
                "change": "change",
                "percentChange": "percent_change",
                "volume": "volume",
                "openInterest": "open_interest",
                "impliedVolatility": "implied_volatility",
                "inTheMoney": "in_the_money",
                "contractSize": "contract_size",
                "currency": "currency"
            })
            
            df = df[["contract_symbol", "ticker", "last_trade_date", "strike", "last_price", "bid", "ask",
                     "change", "percent_change", "volume", "open_interest", "implied_volatility", 
                     "in_the_money", "contract_size", "currency"]]
            
            all_options.append(df)

        final_df = pd.concat(all_options)
        final_df.to_sql("options_data", engine, if_exists="append", index=False)
        print(f"Options data for {ticker} inserted in options_data.")

    except Exception as e:
        print(f"Error al obtener datos de opciones para {ticker}: {e}")


def delete_options_data(ticker):
    engine = connect_db()
    with engine.connect() as conn:
        conn.execute(text("DELETE FROM options_data WHERE ticker = :ticker"), {"ticker": ticker})
        conn.commit()
    print(f"Data from de {ticker} eliminated from options_data.")

# quarterly_financials

In [49]:

def store_quarterly_financials(ticker):
    engine = connect_db()

    stock = yf.Ticker(ticker)
    
    if not hasattr(stock, "quarterly_financials") or stock.quarterly_financials.empty:
        print(f"No se encontraron datos financieros trimestrales para {ticker}.")
        return

    df = stock.quarterly_financials.transpose().reset_index()
    df["date_reported"] = pd.to_datetime(df["index"])
    df["ticker"] = ticker

    df = df.rename(columns={
        "Total Revenue": "total_revenue",
        "Operating Revenue": "operating_revenue",
        "Cost of Revenue": "cost_of_revenue",
        "Gross Profit": "gross_profit",
        "Operating Expense": "operating_expense",
        "Research And Development": "research_and_development",
        "Selling General And Admin": "selling_general_and_admin",
        "Selling And Marketing Expenses": "selling_and_marketing_expenses",
        "General And Admin Expense": "general_and_admin_expense",
        "Other Gand A": "other_gand_a",
        "Operating Income": "operating_income",
        "Interest Income Non Operating": "interest_income_non_operating",
        "Interest Expense Non Operating": "interest_expense_non_operating",
        "Net Non Operating Interest Income": "net_non_operating_interest_income",
        "Gain On Sale Of Security": "gain_on_sale_of_security",
        "Write Off": "write_off",
        "Special Income Charges": "special_income_charges",
        "Other Non Operating Income Expense": "other_non_operating_income_expense",
        "Other Income Expense": "other_income_expense",
        "Pretax Income": "pretax_income",
        "Tax Provision": "tax_provision",
        "Net Income Continuous Operations": "net_income_continuous_operations",
        "Net Income Incl Noncontrolling": "net_income_incl_noncontrolling",
        "Net Income": "net_income",
        "Net Income Common Stockholders": "net_income_common_stockholders",
        "Diluted Ni Availto Com Stockholders": "diluted_ni_avilto_com_stockholders",
        "Basic EPS": "basic_eps",
        "Diluted EPS": "diluted_eps",
        "Basic Avg Shares": "basic_avg_shares",
        "Diluted Avg Shares": "diluted_avg_shares",
        "Total Operating Income": "total_operating_income",
        "Total Expenses": "total_expenses",
        "Net Income Cont Discont Oper": "net_income_cont_discont_oper",
        "Normalized Income": "normalized_income",
        "Interest Income": "interest_income",
        "Interest Expense": "interest_expense",
        "Net Interest Income": "net_interest_income",
        "EBIT": "ebit",
        "EBITDA": "ebitda",
        "Reconciled Cost Of Revenue": "reconciled_cost_of_revenue",
        "Reconciled Depreciation": "reconciled_depreciation",
        "Net Income Cont Operation": "net_income_cont_operation",
        "Total Unusual Items Excl Goodwill": "total_unusual_items_excl_goodwill",
        "Total Unusual Items": "total_unusual_items",
        "Normalized EBITDA": "normalized_ebitda",
        "Tax Rate For Calcs": "tax_rate_for_calcs",
        "Tax Effect Of Unusual Items": "tax_effect_of_unusual_items"
    })

    df = df.drop(columns=["index"])
    
    df.to_sql("quarterly_financials", engine, if_exists="append", index=False)
    print(f"Quarterly financials data for {ticker} inserted in quarterly_financials.")


def delete_quarterly_financials(ticker):
    engine = connect_db()
    with engine.connect() as conn:
        conn.execute(text("DELETE FROM quarterly_financials WHERE ticker = :ticker"), {"ticker": ticker})
        conn.commit()
    print(f"Data from de {ticker} eliminated from quarterly_financials.")


# Get Data

In [50]:
def get_data(ticker):
    print(f"Fetching data for {ticker}...")

    store_historical_prices(ticker)
    print(f"Historical prices for {ticker} stored successfully.")

    store_funds_holding(ticker)
    print(f"Funds holdings for {ticker} stored successfully.")

    store_institutional_shareholders(ticker)
    print(f"Institutional shareholders for {ticker} stored successfully.")

    store_options_data(ticker)
    print(f"Options data for {ticker} stored successfully.")

    store_quarterly_financials(ticker)
    print(f"Quarterly financials for {ticker} stored successfully.")

    store_analyst_recommendations(ticker)
    print(f"Analyst recommendations for {ticker} stored successfully.")

    store_analyst_recommendations_period(ticker)
    print(f"Analyst recommendations period for {ticker} stored successfully.")

    store_company_info(ticker)
    print(f"Company info for {ticker} stored successfully.")

    store_news_data(ticker)
    print(f"News data for {ticker} stored successfully.")

    print(f"All data for {ticker} has been stored successfully.")



In [51]:
def eliminate_data():
    """Vacía por completo todas las tablas de la base de datos."""
    engine = connect_db()
    tables = [
        "historical_prices",
        "funds_holdings",
        "institutional_shareholders",
        "options_data",
        "quarterly_financials",
        "analyst_recommendations",
        "analyst_recommendations_period",
        "company_info",
        "news_data"
    ]

    with engine.connect() as conn:
        for table in tables:
            conn.execute(text(f"DELETE FROM {table}"))
            conn.commit()
            print(f"Table {table} has been emptied.")

    print("All tables have been emptied successfully.")

# Experiment

In [71]:
ticker ='MSFT'

In [72]:
get_data(ticker)

Fetching data for MSFT...


[*********************100%***********************]  1 of 1 completed


DataMSFT inserted in historical_prices.
Historical prices for MSFT stored successfully.
DataMSFT inserted in funds_holding.
Funds holdings for MSFT stored successfully.


KeyError: "['pct_held', 'pct_change'] not in index"

In [73]:
eliminate_data()

Table historical_prices has been emptied.
Table funds_holdings has been emptied.
Table institutional_shareholders has been emptied.
Table options_data has been emptied.
Table quarterly_financials has been emptied.
Table analyst_recommendations has been emptied.
Table analyst_recommendations_period has been emptied.
Table company_info has been emptied.
Table news_data has been emptied.
All tables have been emptied successfully.


# Try historical_prices

In [63]:
engine = connect_db()
    
df = yf.download(ticker, period='5y')

df = df.reset_index()
df["ticker"] = ticker

df = df.rename(columns={"Date": "date", "Open": "open_price", "High": "high_price",
                        "Low": "low_price", "Close": "close_price",
                        "Volume": "volume"})




[*********************100%***********************]  1 of 1 completed


In [ ]:
df.head()

Price,date,close_price,high_price,low_price,open_price,volume,ticker
Ticker,,MSFT,MSFT,MSFT,MSFT,MSFT,
0,2020-02-27,151.406540,159.877578,151.215107,156.326453,93174900,MSFT
1,2020-02-28,155.072479,156.699694,145.491128,145.883575,97012700,MSFT
2,2020-03-02,165.390930,165.515368,155.359700,158.231236,71030800,MSFT
3,2020-03-03,157.465439,167.506247,155.311787,166.357636,71677000,MSFT
4,2020-03-04,163.246841,163.390412,158.527943,161.275056,49814400,MSFT


In [67]:
preprocess_stock_data(df)

,date,close_price,high_price,low_price,open_price,volume,ticker
0,2020-02-27,151.406540,159.877578,151.215107,156.326453,93174900,MSFT
1,2020-02-28,155.072479,156.699694,145.491128,145.883575,97012700,MSFT
2,2020-03-02,165.390930,165.515368,155.359700,158.231236,71030800,MSFT
3,2020-03-03,157.465439,167.506247,155.311787,166.357636,71677000,MSFT
4,2020-03-04,163.246841,163.390412,158.527943,161.275056,49814400,MSFT
...,...,...,...,...,...,...,...
1253,2025-02-21,408.209991,418.049988,407.890015,417.339996,27524800,MSFT
1254,2025-02-24,404.000000,409.369995,399.320007,408.510010,26443700,MSFT
1255,2025-02-25,397.899994,401.920013,396.700012,401.100006,29387400,MSFT
1256,2025-02-26,399.730011,403.600006,394.250000,398.010010,19619000,MSFT


In [68]:
df.to_sql("historical_prices", engine, if_exists="append", index=False)


print(f"Data{ticker} inserted in historical_prices.")

DataMSFT inserted in historical_prices.


## Delete

In [70]:
delete_historical_prices(ticker)

Datos históricos de MSFT eliminados de historical_prices.


# Try with the rest if needed and fix columns